In [1]:
import pandas as pd
import requests, json
import numpy as np

# Пиздим инфу

In [2]:
# Запрос для получения json
headers = '''
Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://auto.ru/cars/bmw/all/?page=3
x-client-app-version: bce8a8244b5
x-page-request-id: 328918ce0247597eaf5327bfb58fbbf0
x-client-date: 1621052424675
x-csrf-token: 46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d
x-requested-with: fetch
content-type: application/json
Origin: https://auto.ru
Content-Length: 78
Connection: keep-alive
Cookie: autoru_sid=a%3Ag609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2%7C1620997898875.604800.wyDgKeg1mlEd3UgRUP-bJA.67jb8JxaCRrLGKWbo9PTOIBxTdPMTL7-ErqZR1uTZ90; autoruuid=g609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2; autoru_gdpr=1; suid=c18f94687848ae634a3e33b24dec6d0a.4fc40ee98cea6dd87fe788cb17ac994d; yuidlt=1; yandexuid=3854193761620997913; counter_ga_all7=2; crookie=An+ejc3GId86JbaHno+3yDpWbPhSpPl+/G5iLmug8RDpM4VJDMjLzHii5iDi/5MZT+TrlHvnUdD/Khj0WeuIogIWnpE=; cmtchd=MTYyMDk5NzkyNzY0NA==; _ym_uid=162099793885069073; _ym_d=1621052333; _ym_isad=2; cycada=1+Z/guXKzEVROtphAKfMJ/7TQUH18nTGnu0U0PuR/rk=; _ga=GA1.2.1678141264.1620998016; _gid=GA1.2.1513849119.1620998016; parts_ur=; _csrf_token=46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d; from_lifetime=1621052333763; from=direct; X-Vertis-DC=vla; gdpr=0
Pragma: no-cache
Cache-Control: no-cache
'''.strip().split("\n")

# Преоб. запроса в словарь
dict_header = {}
for header in headers:
    key, value = header.split(': ')
    dict_header[key] = value
print(dict_header)

{'Host': 'auto.ru', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0', 'Accept': '*/*', 'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3', 'Accept-Encoding': 'gzip, deflate, br', 'Referer': 'https://auto.ru/cars/bmw/all/?page=3', 'x-client-app-version': 'bce8a8244b5', 'x-page-request-id': '328918ce0247597eaf5327bfb58fbbf0', 'x-client-date': '1621052424675', 'x-csrf-token': '46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d', 'x-requested-with': 'fetch', 'content-type': 'application/json', 'Origin': 'https://auto.ru', 'Content-Length': '78', 'Connection': 'keep-alive', 'Cookie': 'autoru_sid=a%3Ag609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2%7C1620997898875.604800.wyDgKeg1mlEd3UgRUP-bJA.67jb8JxaCRrLGKWbo9PTOIBxTdPMTL7-ErqZR1uTZ90; autoruuid=g609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2; autoru_gdpr=1; suid=c18f94687848ae634a3e33b24dec6d0a.4fc40ee98cea6dd87fe788cb17ac994d; yuidlt=1; yandexuid=3854193

In [3]:
# адрес спизженный из мозилы, где находится json file
url = "https://auto.ru/-/ajax/desktop/listing/"

In [4]:
def parser(model, page_range):
    offers = []
    fail = []
    try:
        for x in range(1, page_range):
            param = {"color": ["FFD600", "660099", "DEA522", "007F00", "040001", "C49648", "CACECB", "EE1D19", "0000CC", "22A0F8", "FFC0CB", "4A2197", "FF8649", "200204", "97948F", "FAFBFB"],
                     "year_from": 1900, 
                     "year_to": 2021, 
                     "catalog_filter": [{"mark":model}], 
                     "damage_group": "ANY", 
                     'sort': "fresh_relevance_1-desc", #
                     "top_days":"900",#
                     "customs_state_group": "DOESNT_MATTER", 
                     "section": "all", 
                     "category": "cars",
                     "page":x
                    }
#             param = {"category":"cars",
#                      "section":"all",
#                      "page":x,
#                      "catalog_filter":[{"mark":model}]}
            response = requests.post(
                url, json=param, headers=dict_header)  # тырим json
            print(response.status_code)  # Статус запроса
            data = response.json()  # извлекаем стыренное
            # берем из json только оферы на тачки. Мусор не берем
            offers.extend(data['offers'])
            print(f'Current_page: {x}')  # Стата по страницам
    except:
        fail.apend(model)
        print('Fail')
    with open(f'data_{model}.json', 'w') as f:
        json.dump(offers, f)
    print(f'File "data_{model}" is ready')
    return fail

In [5]:
brand_list = ['BMW',
              'VOLKSWAGEN',
              'NISSAN',
              'MERCEDES',
              'TOYOTA',
              'AUDI',
              'MITSUBISHI',
              'SKODA',
              'VOLVO',
              'HONDA',
              'INFINITI',
              'LEXUS','FORD','KIA','HYNDAI'
]

In [6]:
for i in brand_list:
    parser(i,200)

200
Current_page: 1
200
Current_page: 2
200
Current_page: 3
200
Current_page: 4
200
Current_page: 5
200
Current_page: 6
200
Current_page: 7
200
Current_page: 8
200
Current_page: 9
200
Current_page: 10
200
Current_page: 11
200
Current_page: 12
503
Fail
File "data_BMW" is ready
200
Current_page: 1
200
Current_page: 2
200
Current_page: 3
200
Current_page: 4
200
Current_page: 5
200
Current_page: 6
200
Current_page: 7
200
Current_page: 8
200
Current_page: 9
200
Current_page: 10
200
Current_page: 11
200
Current_page: 12
200
Current_page: 13
200
Current_page: 14
200
Current_page: 15
200
Current_page: 16
200
Current_page: 17
200
Current_page: 18
200
Current_page: 19
200
Current_page: 20
200
Current_page: 21
200
Current_page: 22
200
Current_page: 23
200
Current_page: 24
200
Current_page: 25
200
Current_page: 26
200
Current_page: 27
200
Current_page: 28
200
Current_page: 29
200
Current_page: 30
200
Current_page: 31
200
Current_page: 32
200
Current_page: 33
200
Current_page: 34
200
Current_page: 

200
Current_page: 171
200
Current_page: 172
200
Current_page: 173
200
Current_page: 174
200
Current_page: 175
200
Current_page: 176
200
Current_page: 177
200
Current_page: 178
200
Current_page: 179
200
Current_page: 180
200
Current_page: 181
200
Current_page: 182
200
Current_page: 183
200
Current_page: 184
200
Current_page: 185
200
Current_page: 186
200
Current_page: 187
200
Current_page: 188
200
Current_page: 189
200
Current_page: 190
200
Current_page: 191
200
Current_page: 192
200
Current_page: 193
200
Current_page: 194
200
Current_page: 195
200
Current_page: 196
200
Current_page: 197
200
Current_page: 198
200
Current_page: 199
File "data_NISSAN" is ready
200
Current_page: 1
200
Current_page: 2
200
Current_page: 3
200
Current_page: 4
200
Current_page: 5
200
Current_page: 6
200
Current_page: 7
200
Current_page: 8
200
Current_page: 9
200
Current_page: 10
200
Current_page: 11
200
Current_page: 12
200
Current_page: 13
200
Current_page: 14
200
Current_page: 15
200
Current_page: 16
200
Cur

200
Current_page: 54
200
Current_page: 55
200
Current_page: 56
200
Current_page: 57
200
Current_page: 58
200
Current_page: 59
200
Current_page: 60
200
Current_page: 61
200
Current_page: 62
200
Current_page: 63
200
Current_page: 64
200
Current_page: 65
200
Current_page: 66
200
Current_page: 67
200
Current_page: 68
200
Current_page: 69
200
Current_page: 70
200
Current_page: 71
200
Current_page: 72
200
Current_page: 73
200
Current_page: 74
200
Current_page: 75
200
Current_page: 76
200
Current_page: 77
200
Current_page: 78
200
Current_page: 79
200
Current_page: 80
200
Current_page: 81
200
Current_page: 82
200
Current_page: 83
200
Current_page: 84
200
Current_page: 85
200
Current_page: 86
200
Current_page: 87
200
Current_page: 88
200
Current_page: 89
200
Current_page: 90
200
Current_page: 91
200
Current_page: 92
200
Current_page: 93
200
Current_page: 94
200
Current_page: 95
200
Current_page: 96
200
Current_page: 97
200
Current_page: 98
200
Current_page: 99
200
Current_page: 100
200
Current_

200
Current_page: 35
200
Current_page: 36
200
Current_page: 37
200
Current_page: 38
200
Current_page: 39
200
Current_page: 40
200
Current_page: 41
200
Current_page: 42
200
Current_page: 43
200
Current_page: 44
200
Current_page: 45
200
Current_page: 46
200
Current_page: 47
200
Current_page: 48
200
Current_page: 49
200
Current_page: 50
200
Current_page: 51
200
Current_page: 52
200
Current_page: 53
200
Current_page: 54
200
Current_page: 55
200
Current_page: 56
200
Current_page: 57
200
Current_page: 58
200
Current_page: 59
200
Current_page: 60
200
Current_page: 61
200
Current_page: 62
200
Current_page: 63
200
Current_page: 64
200
Current_page: 65
200
Current_page: 66
200
Current_page: 67
200
Current_page: 68
200
Current_page: 69
200
Current_page: 70
200
Current_page: 71
200
Current_page: 72
200
Current_page: 73
200
Current_page: 74
200
Current_page: 75
200
Current_page: 76
200
Current_page: 77
200
Current_page: 78
200
Current_page: 79
200
Current_page: 80
200
Current_page: 81
200
Current_p

200
Current_page: 16
200
Current_page: 17
200
Current_page: 18
200
Current_page: 19
200
Current_page: 20
200
Current_page: 21
200
Current_page: 22
200
Current_page: 23
200
Current_page: 24
200
Current_page: 25
200
Current_page: 26
200
Current_page: 27
200
Current_page: 28
200
Current_page: 29
200
Current_page: 30
200
Current_page: 31
200
Current_page: 32
200
Current_page: 33
200
Current_page: 34
200
Current_page: 35
200
Current_page: 36
200
Current_page: 37
200
Current_page: 38
200
Current_page: 39
200
Current_page: 40
200
Current_page: 41
200
Current_page: 42
200
Current_page: 43
200
Current_page: 44
200
Current_page: 45
200
Current_page: 46
200
Current_page: 47
200
Current_page: 48
200
Current_page: 49
200
Current_page: 50
200
Current_page: 51
200
Current_page: 52
200
Current_page: 53
200
Current_page: 54
200
Current_page: 55
200
Current_page: 56
200
Current_page: 57
200
Current_page: 58
200
Current_page: 59
200
Current_page: 60
200
Current_page: 61
200
Current_page: 62
200
Current_p

200
Current_page: 197
200
Current_page: 198
200
Current_page: 199
File "data_INFINITI" is ready
200
Current_page: 1
200
Current_page: 2
200
Current_page: 3
200
Current_page: 4
200
Current_page: 5
200
Current_page: 6
200
Current_page: 7
200
Current_page: 8
200
Current_page: 9
200
Current_page: 10
200
Current_page: 11
200
Current_page: 12
200
Current_page: 13
200
Current_page: 14
200
Current_page: 15
200
Current_page: 16
200
Current_page: 17
200
Current_page: 18
200
Current_page: 19
200
Current_page: 20
200
Current_page: 21
200
Current_page: 22
200
Current_page: 23
200
Current_page: 24
200
Current_page: 25
200
Current_page: 26
200
Current_page: 27
200
Current_page: 28
200
Current_page: 29
200
Current_page: 30
200
Current_page: 31
200
Current_page: 32
200
Current_page: 33
200
Current_page: 34
200
Current_page: 35
200
Current_page: 36
200
Current_page: 37
200
Current_page: 38
200
Current_page: 39
200
Current_page: 40
200
Current_page: 41
200
Current_page: 42
200
Current_page: 43
200
Curren

200
Current_page: 179
200
Current_page: 180
200
Current_page: 181
200
Current_page: 182
200
Current_page: 183
200
Current_page: 184
200
Current_page: 185
200
Current_page: 186
200
Current_page: 187
200
Current_page: 188
200
Current_page: 189
200
Current_page: 190
200
Current_page: 191
200
Current_page: 192
200
Current_page: 193
200
Current_page: 194
200
Current_page: 195
200
Current_page: 196
200
Current_page: 197
200
Current_page: 198
200
Current_page: 199
File "data_FORD" is ready
200
Current_page: 1
200
Current_page: 2
200
Current_page: 3
200
Current_page: 4
200
Current_page: 5
200
Current_page: 6
200
Current_page: 7
200
Current_page: 8
200
Current_page: 9
200
Current_page: 10
200
Current_page: 11
200
Current_page: 12
200
Current_page: 13
200
Current_page: 14
200
Current_page: 15
200
Current_page: 16
200
Current_page: 17
200
Current_page: 18
200
Current_page: 19
200
Current_page: 20
200
Current_page: 21
200
Current_page: 22
200
Current_page: 23
200
Current_page: 24
200
Current_page:

# Обрабатываем спизженное

In [10]:
cols = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня','price_segment','price']

In [11]:
def get_df(parsed_json):
    df = pd.DataFrame(data=None,columns=cols,index = range(len(parsed_json)))
    NaN = np.nan
    for i, vehicle in enumerate(parsed_json):
        try:
            df.loc[i]['color'] = vehicle['color_hex']
        except:
            df.loc[i]['color'] = NaN
        try:
            df.loc[i].brand = vehicle['vehicle_info']['mark_info']['name']
        except:
            df.loc[i].brand = NaN
        try:
            df.loc[i].complectation_dict = vehicle['vehicle_info']['complectation']
        except:
            df.loc[i].complectation_dict = NaN
        try:
            df.loc[i].engineDisplacement = vehicle['vehicle_info']['tech_param']['displacement']
        except:
            df.loc[i].engineDisplacement = NaN
        try:
            df.loc[i].enginePower = vehicle['vehicle_info']['tech_param']['power']
        except:
            df.loc[i].enginePower = NaN
        try:
            df.loc[i].equipment_dict = vehicle['vehicle_info']['equipment']
        except:
            df.loc[i].equipment_dict = NaN
        try:
            df.loc[i].fuelType = vehicle['lk_summary'].split(',')[-1]
        except:
            df.loc[i].fuelType = NaN
        try:
            df.loc[i].image = vehicle['state']['image_urls'][0]['sizes']['1200x900n']
        except:
            df.loc[i].image = NaN
        try:
            df.loc[i].mileage = vehicle['state']['mileage']
        except:
            df.loc[i].mileage = NaN
        try:
            df.loc[i].modelDate = vehicle['vehicle_info']['super_gen']['year_from']
        except:
            df.loc[i].modelDate = NaN
        try:
            df.loc[i].model_info = vehicle['vehicle_info']['model_info']
        except:
            df.loc[i].model_info = NaN
        try:
            df.loc[i].model_name = vehicle['vehicle_info']['model_info']['code']
        except:
            df.loc[i].model_name = NaN
        try:
            df.loc[i].name = vehicle['vehicle_info']['tech_param']['human_name']
#             df.loc[i]['name'] = vehicle['lk_summary']  # !!!!!!!!!!!!!!!!!!!
        except:
            df.loc[i]['name'] = NaN
        try:
            df.loc[i].numberOfDoors = vehicle['vehicle_info']['configuration']['doors_count']
        except:
            df.loc[i].numberOfDoors = NaN
        try:
            df.loc[i].parsing_unixtime = 'today'
        except:
            df.loc[i].parsing_unixtime = NaN
        try:
            df.loc[i].productionDate = vehicle['documents']['year']
        except:
            df.loc[i].productionDate = NaN
        try:
            df.loc[i].sell_id = vehicle['id']
        except:
            df.loc[i].sell_id = NaN
        try:
            df.loc[i].super_gen = vehicle['vehicle_info']['super_gen']
        except:
            df.loc[i].super_gen = NaN
        try:
            df.loc[i].vehicleConfiguration = 'useless_col'
        except:
            df.loc[i].vehicleConfiguration = NaN
        try:
            df.loc[i].vehicleTransmission = vehicle['vehicle_info']['tech_param']['transmission']
        except:
            df.loc[i].vehicleTransmission = NaN
        try:
            df.loc[i].vendor = vehicle['vehicle_info']['vendor']
        except:
            df.loc[i].vendor = NaN
        try:
            df.loc[i].Владение = 'useless_col'
        except:
            df.loc[i].Владение = NaN
        try:
            df.loc[i].Привод = vehicle['lk_summary'].split(',')[2]
        except:
            df.loc[i].Привод = NaN
        try:
            df.loc[i].Состояние = vehicle['state']['state_not_beaten']
        except:
            df.loc[i].Состояние = NaN
        try:
            df.loc[i].Таможня = vehicle['documents']['custom_cleared']
        except:
            df.loc[i].Таможня = NaN
        try:
            df.loc[i].price_segment = vehicle['vehicle_info']['super_gen']['price_segment']
        except:
            df.loc[i].price_segmen = NaN
        try:
            df.loc[i].bodyType = vehicle['vehicle_info']['configuration']['human_name']
        except:
            df.loc[i].bodyType = NaN
        try:
            df.loc[i].description = vehicle['description']
        except:
            df.loc[i].description = NaN
        try:
            df.loc[i].Владельцы = vehicle['documents']['owners_number']
        except:
            df.loc[i].Владельцы = NaN
        try:
            df.loc[i].price = vehicle['price_info']['price']
        except:
            df.loc[i].price = NaN
        try:
            df.loc[i].priceCurrency = vehicle['price_info']['currency']
        except:
            df.loc[i].priceCurrency = NaN
        try:
            df.loc[i].ПТС = vehicle['documents']['pts']
        except:
            df.loc[i].ПТС = NaN
        try:
            df.loc[i].Руль = vehicle['vehicle_info']['steering_wheel']
        except:
            df.loc[i].Руль = NaN
            
    return df

In [12]:
def get_train(car_name):
    df = pd.DataFrame(data=None)
    with open (f'data_{car_name}.json', 'r', encoding='utf8') as f:
        data = json.load(f)
    df = get_df(data)
    return df

In [ ]:
AUDI = get_train('AUDI')
BMW = get_train('BMW')
HONDA = get_train('HONDA')
INFINITI = get_train('INFINITI')
LEXUS = get_train('LEXUS')
MERCEDES = get_train('MERCEDES')
MITSUBISHI = get_train('MITSUBISHI')
NISSAN = get_train('NISSAN')
SKODA = get_train('SKODA')
TOYOTA = get_train('TOYOTA')
VOLKSWAGEN = get_train('VOLKSWAGEN')
FORD = get_train('FORD')
HYNDAI = get_train('HYNDAI')
KIA = get_train('KIA')
VOLVO = get_train('VOLVO')


In [ ]:
df_list = [BMW,VOLKSWAGEN,
          NISSAN,MERCEDES,
          TOYOTA,AUDI,
          MITSUBISHI,SKODA,
          VOLVO,HONDA,
          INFINITI,LEXUS,FORD,HYNDAI,KIA
          ]
df = pd.concat(df_list, sort=False)

In [ ]:
colors = {'FFD600':'Жёлтый',
          "660099":'Пурпурный',
          "DEA522":'Золотистый',
          "007F00":'Зелёный',
          "040001":'Чёрный',
          "C49648":'Бежевый',
          "CACECB":'Серебристый',
          "EE1D19":'Красный',
          "0000CC":'Синий',
          "22A0F8":'Голубой',
          "FFC0CB":'Розовый',
          "4A2197":'Фиолетовый',
          "FF8649":'Оранжевый',
          "200204":'Коричневый',
          "97948F":'Серый',
          "FAFBFB":'Белый'}

In [ ]:
df['color'] = df['color'].map(colors)

In [ ]:
df.to_csv('train.csv', index = False)

In [ ]:
with open (f'data_AUDI.json', 'r', encoding='utf8') as f:
    data = json.load(f)

In [ ]:
for i in data:
    print(i['vehicle_info']['tech_param']['human_name'])